In [1]:
import numpy as np
import pandas as pd
%matplotlib notebook

## 12.1 分类数据

### 12.1.1 背景和目的

表中的一列通常会有重复的包含不同值的小集合的情况

In [4]:
values = pd.Series(['apple', 'orange', 'apple', 'apple']*2)
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [5]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [6]:
pd.value_counts(values)

apple     6
orange    2
dtype: int64

在数据仓库中，表征重复值的最好方法是使用包含不同值的维表(Dimension Table)，将主要的参数存储为引用维表整数键

In [8]:
values = pd.Series([0, 1, 0, 0]*2)
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [9]:
dim = pd.Series(['apple', 'orange'])

In [10]:
dim

0     apple
1    orange
dtype: object

使用 instance.take() 存储原始的字符串 Series

In [11]:
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

### 12.1.2 pandas 中的 Categorical 类型

pandas 有一个特殊的分类类型，用于保存使用整数分类表示法的数据。

In [2]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2

In [3]:
N = len(fruits)

In [4]:
df = pd.DataFrame({'fruit': fruits,
                  'basket_id': np.arange(N),
                  'count': np.random.randint(3, 15, size=N),
                  'weight': np.random.uniform(0, 4, size=N)},
                 columns=['basket_id', 'fruit', 'count', 'weight'])

In [5]:
df

,basket_id,fruit,count,weight
0,0,apple,7,1.447788
1,1,orange,4,3.228978
2,2,apple,13,0.699826
3,3,apple,13,3.325490
4,4,apple,3,3.130308
5,5,orange,6,0.147423
6,6,apple,10,2.125430
7,7,apple,12,3.515150


这里，df['fruit'] 是一个 Python 字符串对象数组。可以将它转变为分类

In [21]:
df['fruit']

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: object

调用 astype 将它转换为 Categorical 对象

In [6]:
fruit_cat = df['fruit'].astype('category')

In [7]:
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

fruit_cat 的值不是 NunPy 数组，而是一个 pandas.Categorical 实例

In [22]:
c = fruit_cat.values

In [23]:
type(c)

pandas.core.arrays.categorical.Categorical

分类对象有 categories and codes 属性

In [24]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [25]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

可将 DataFrame 的列通过分配转换结果，转换为分类

In [26]:
df['fruit'] = df['fruit'].astype('category')

In [27]:
df.fruit

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

还可从其它 Python 序列直接创建 pandas.Categorical

In [28]:
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])

In [29]:
my_categories

[foo, bar, baz, foo, bar]
Categories (3, object): [bar, baz, foo]

如果已经从其它源获得了分类编码，还可以使用 from_codes 构造器

In [30]:
categories = ['foo', 'bar', 'baz']

In [31]:
codes = [0, 1, 2, 0, 0, 1]

In [32]:
my_cats_2 = pd.Categorical.from_codes(codes, categories)

In [34]:
my_cats_2

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo, bar, baz]

输入数据的顺序不同，则 categories 数组的顺序会相应改变  
当使用 from_codes 或其它的构造器时，可以指定分类一个有意义的顺序

In [35]:
ordered_cat = pd.Categorical.from_codes(codes, categories,
                                       ordered=True)

In [37]:
# 输出 [foo<bar<baz] 指明 'foo' 位于 'bar' 的前面

ordered_cat

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

无序的分类实例可以通过 as_ordered 排序

In [38]:
my_cats_2.as_ordered()

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

### 12.1.3 使用 Categorical 对象进行计算

In [39]:
np.random.seed(12345)

In [40]:
draws = np.random.randn(1000)

In [41]:
draws[ : 5]

array([-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057])

计算这个的分位面元，提取一些统计信息

In [42]:
bins = pd.qcut(draws, 4)

In [43]:
bins

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.9499999999999997, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]

以上，确切的样本分位数与分位的名称相比，不利于生成汇总。  
可以使用 labels 参数 qcut，实现目的

In [44]:
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])

In [45]:
bins

[Q2, Q3, Q2, Q2, Q4, ..., Q3, Q2, Q1, Q3, Q4]
Length: 1000
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

In [46]:
bins.codes[ : 10]

array([1, 2, 1, 1, 3, 3, 2, 2, 3, 3], dtype=int8)

加上标签的面元分类不包含数据面元边界的信息，因此可以使用 groupby 提取一些汇总信息

In [47]:
bins = pd.Series(bins, name='quartile')

In [48]:
results = (pd.Series(draws)
          .groupby(bins)
          .agg(['count', 'min', 'max'])
          .reset_index())

In [50]:
results

,quartile,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


分位数保存了原始的面元分类信息，包括排序

In [51]:
results['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

### 12.1.4 用分类提高性能

如果在一个特定数据集上做大量分析，将其转换为分类可以极大地提高效率  
DataFrame 列的分类使用的内存通常少得多

一些包含一千万元素的 Series，和一些不同的分类

In [71]:
N = 10000000

In [72]:
draws = pd.Series(np.random.randn(N))

In [73]:
labels = pd.Series(['foo', 'bar', 'baz', 'qux']) * (N // 4)

现在，将标签转换为分类

In [74]:
categories = labels.astype('category')

这是，可以看到标签使用的内存远比分类多

In [75]:
labels.memory_usage()

112

In [76]:
categories.memory_usage()

276

转换为分类不是没有代价的，但这是一次性的代价

### 12.1.5 分类方法

包含分类数据的 Series 有一些特殊的方法，类似于 Series.str 字符串方法。  
它还提供了方便的分类和编码的使用方法

In [77]:
s = pd.Series(['a', 'b', 'c',  'd']*2)

In [78]:
cat_s = s.astype('category')

In [79]:
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

特别的 cat 属性提供了分类方法的入口

In [80]:
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [81]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

假设知道这个数据的实际分类集，超出了数据中的四个值。  
可以使用 set_categories 方法改变它们

In [82]:
actual_categories = ['a', 'b', 'c', 'd', 'e']

In [83]:
cat_s2 = cat_s.cat.set_categories(actual_categories)

In [84]:
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): [a, b, c, d, e]

虽然数据看起来没变，新的分类将反映在它们的操作中。  
例如，如果有的话，value_counts 表示分类

In [85]:
cat_s.value_counts()

d    2
c    2
b    2
a    2
dtype: int64

In [86]:
cat_s2.value_counts()

d    2
c    2
b    2
a    2
e    0
dtype: int64

在大数据集中，分类经常作为节省内存和高性能的便捷工具  
过滤完大 DataFrame 和 Series 之后，许多分类可能不会出现在数据中。  
可以使用 remove_unused_categories 方法删除没看到的分类

In [87]:
cat_s3 = cat_s[cat_s.isin(['a', 'b'])]

In [88]:
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): [a, b, c, d]

In [89]:
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): [a, b]

In [91]:
%%html
<img src="picture/表12-1 pandas的Series的分类方法.png", width=600>

### 12.1.6 为建模创建虚拟变量

当你使用统计或机器学习工具时，通常会将分类数据转换为虚拟变量，也称为one-hot编码。这包括创建一个不同类别的列的DataFrame；这些列包含给定分类的1s，其它为0。

In [92]:
cat_s = pd.Series(['a', 'b', 'c', 'd']*2, dtype='category')

pandas.get_dummies 函数可以转换这个分类数据为包含虚拟变量的 DataFrame

In [93]:
pd.get_dummies(cat_s)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


## 12.2 GroupBy 高级应用

### 12.2.1 分组转换和“解封” GroupBy

在第10章，我们在分组操作中学习了apply方法，进行转换。还有另一个transform方法，它与apply很像，但是对使用的函数有一定限制：  
- 它可以产生向分组形状广播标量值
- 它可以产生一个和输入组形状相同的对象
- 它不能修改输入

In [96]:
df = pd.DataFrame({'key': ['a', 'b', 'c']*4,
                  'value': np.arange(12.)})

In [97]:
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


按键进行分组

In [98]:
g = df.groupby('key').value

In [99]:
g

In [100]:
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

假设我们想产生一个和df['value']形状相同的Series，但值替换为按键分组的平均值。我们可以传递函数lambda x: x.mean()进行转换

In [101]:
g.transform(lambda x: x.mean())

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

对于内置的聚合函数，可以传递一个字符串假名作为 GroupBy 的 agg 方法

In [103]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

与 apply 类似，transform 的函数会返回 Series，但是结果必须与输入大小相同

In [104]:
g.transform(lambda x: x*2)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

计算每个分组的降序排名

In [106]:
g.transform(lambda x: x.rank(ascending=False))

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

一个由简单聚合构造的分组转换函数

In [107]:
def normalize(x):
    return (x - x.mean()) / x.std()

用 transform or apply 可以获得等价的结果

In [108]:
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [109]:
g.apply(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

内置的聚合函数，比如 mean or sum，通过比 apply 函数快，也比 transform 快  
这允许我们进行一个解封(unwrapped)分组操作

In [110]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [111]:
normalize = (df['value'] - g.transform('mean') / g.transform('std'))

In [112]:
normalize

0    -1.161895
1    -0.420094
2     0.321707
3     1.838105
4     2.579906
5     3.321707
6     4.838105
7     5.579906
8     6.321707
9     7.838105
10    8.579906
11    9.321707
Name: value, dtype: float64

### 12.2.2 分组的时间重采样

对于时间序列数据，resample方法从语言上是一个基于内在时间的分组操作

In [113]:
N = 15

In [116]:
times = pd.date_range('2017-05-20 00:00', freq='1min', periods=N)

In [117]:
df = pd.DataFrame({'time': times,
                  'value': np.arange(N)})

In [118]:
df

,time,value
0,2017-05-20 00:00:00,0
1,2017-05-20 00:01:00,1
2,2017-05-20 00:02:00,2
3,2017-05-20 00:03:00,3
4,2017-05-20 00:04:00,4
5,2017-05-20 00:05:00,5
6,2017-05-20 00:06:00,6
7,2017-05-20 00:07:00,7
8,2017-05-20 00:08:00,8
9,2017-05-20 00:09:00,9


用 time 作为索引，然后重采样

In [120]:
df.set_index('time').resample('5min').count()

,value
time,
2017-05-20 00:00:00,5
2017-05-20 00:05:00,5
2017-05-20 00:10:00,5


假设 DataFrame 包含多个时间序列，用一个额外的分组键的列进行标记

In [122]:
df2 = pd.DataFrame({'time': times.repeat(3),
                   'key': np.tile(['a', 'b', 'c'], N),
                   'value': np.arange(N * 3.)})

In [123]:
df2[ : 7]

,time,key,value
0,2017-05-20 00:00:00,a,0.0
1,2017-05-20 00:00:00,b,1.0
2,2017-05-20 00:00:00,c,2.0
3,2017-05-20 00:01:00,a,3.0
4,2017-05-20 00:01:00,b,4.0
5,2017-05-20 00:01:00,c,5.0
6,2017-05-20 00:02:00,a,6.0


要对每个 key 值进行相同的重采样，可引入 pandas.TimeGrouper 对象

In [136]:
time_key = pd.TimeGrouper("5min")

d:\Users\liu59\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: pd.TimeGrouper is deprecated and will be removed; Please use pd.Grouper(freq=...)
  """Entry point for launching an IPython kernel.


设定时间索引，用 key and time_key 分组，然后聚合

In [137]:
resampled = (df2.set_index('time')
           .groupby(['key', time_key])
           .sum())

In [138]:
resampled

value
key time                      
a   2017-05-20 00:00:00   30.0
    2017-05-20 00:05:00  105.0
    2017-05-20 00:10:00  180.0
b   2017-05-20 00:00:00   35.0
    2017-05-20 00:05:00  110.0
    2017-05-20 00:10:00  185.0
c   2017-05-20 00:00:00   40.0
    2017-05-20 00:05:00  115.0
    2017-05-20 00:10:00  190.0

## 12.3 链式编程技术